<a href="https://colab.research.google.com/github/jinyingtld/python/blob/main/AI6126_tutorial7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image Editing 
In this tutorial, we will get to know:
* basic knowledge regarding image-to-image generation and image style transfer.
* two classic image-to-image translation methods - pix2pix (with paired training data) and CycleGAN (with unpaired training data).
* train pix2pix and CycleGAN
* perform inference with a pretrained models.




## PART I Image-to-Image Translation

### Introduction
Let's first recap some background knowledge about image-to-image translation.
Image-to-image translation usually synthesizes images from another set of images. Image-to-image translation tasks can also be divided into two types: **paired** and **unpaired**. The former usually contains paired image data in the training set therefore is easier for image synthesis; the latter does not need paired training data hence is more challenging.

In this tutorial, we will focus on two methods, pix2pix for paired image-to-image translation and CycleGAN for unpaired image-to-image translation, to learn how to generate high-fidelity images. 

Code reference [Jun-Yan Zhu](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix)

## Requirement 
* Pytorch 
* torchvision
* opencv
* matplotlib
* pillow

Google Collab has install these packages

In [3]:
# Check nvcc version 
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [7]:
import os
from PIL import Image 
import matplotlib.pyplot as plt 

import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np
from datetime import datetime 
import time
import logging
import random
import cv2
import math 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import torch.autograd as autograd
from torch.autograd import Variable
from torch.nn import Linear, Conv2d, BatchNorm2d, LayerNorm, LeakyReLU, ConvTranspose2d, ReLU, Tanh, InstanceNorm2d
from torch.nn import ReflectionPad2d, ReplicationPad2d

from torch.utils.data import DataLoader
import torchvision.datasets as dset 
import torchvision 
import itertools
from tqdm import tqdm

In [8]:
# Check Pytorch installation 
print(torch.__version__, torch.cuda.is_available())

1.10.0+cu111 True


In [9]:
# Show the current folder
!ls

sample_data


## useful functions

In [ ]:
# view images 
def visualize(img_arr, dpi=80):
    plt.figure(figsize=(10,10), dpi=dpi)
    plt.imshow(((img_arr.numpy().transpose(1,2,0) + 1.0)*127.5).astype(np.uint8))
    plt.axis('off')
    plt.show()

# load one image in tensor format 
# transform images into 1*C*H*W; data range [-1,1]
def load_image(filename):
    